Following https://www.tensorflow.org/tutorials/images/transfer_learning tutorial

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import importlib
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# Step 1: Loading in Dataset
ds_directory = '/home/zrarnz/9-Projects/ECET380/Fish_Dataset/fishset2'
os.listdir(ds_directory)

['Sea Bass',
 'Trout',
 'Striped Red Mullet',
 'README.txt',
 'Hourse Mackerel',
 'Red Sea Bream',
 'Black Sea Sprat',
 'Shrimp',
 'Gilt-Head Bream',
 'Red Mullet',
 'Segmentation_example_script.m',
 'license.txt']

In [3]:
# Load in ordered dataset
BATCH_SIZE = 16
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)
dataset = image_dataset_from_directory(ds_directory,
                        shuffle=True,
                        batch_size=BATCH_SIZE,
                        image_size=IMG_SIZE,
                        labels="inferred",
                        label_mode='int')


Found 9000 files belonging to 9 classes.


In [4]:
dataset.class_names

['Black Sea Sprat',
 'Gilt-Head Bream',
 'Hourse Mackerel',
 'Red Mullet',
 'Red Sea Bream',
 'Sea Bass',
 'Shrimp',
 'Striped Red Mullet',
 'Trout']

In [5]:
DATASET_SIZE = 9000

In [6]:
train_size = int(0.7 * float(DATASET_SIZE / BATCH_SIZE))
val_size = int(0.15 * float(DATASET_SIZE / BATCH_SIZE))
test_size = int(0.15 * float(DATASET_SIZE / BATCH_SIZE))

# dataset = dataset.shuffle(1000)
train_set = dataset.take(train_size)
test_set = dataset.skip(train_size)
val_set = test_set.take(val_size)
test_set = test_set.skip(val_size)

print(train_size, val_size, test_size)

393 84 84


In [7]:
batch_size = 16
def preprocess(image, label):
    # resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.efficientnet.preprocess_input(image)
    return final_image, label
train_set = train_set.shuffle(1000).repeat(2)
train_set = train_set.map(preprocess).prefetch(1)
val_set = val_set.map(preprocess).prefetch(1)
test_set = test_set.map(preprocess).prefetch(1)

In [8]:
base_model = tf.keras.applications.EfficientNetB0(input_shape=IMG_SHAPE,
                                                  input_tensor=None,
                                               include_top=False,
                                               pooling=None,
                                                classes=1000,
                                               weights='imagenet')

In [9]:
print('Number of dataset batches: %d' % tf.data.experimental.cardinality(dataset))
print('Number of fullset batches: %d' % tf.data.experimental.cardinality(train_set))


print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_set))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_set))

Number of dataset batches: 563
Number of fullset batches: 786
Number of validation batches: 84
Number of test batches: 86


In [10]:
for layer in base_model.layers:
    layer.trainable = False

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(
    units = 9, 
    activation = "softmax")(avg)

model = keras.models.Model(
    inputs = base_model.input,
    outputs = output)

model.compile(
    optimizer = keras.optimizers.SGD(
        learning_rate = 0.01,
        ),
    loss = "sparse_categorical_crossentropy", 
    metrics = ['accuracy'])

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [11]:
# tuner = kt.RandomSearch(
#             model_builder,
#             objective='val_accuracy',
#             max_trials=27,
#             seed=0,
#             overwrite=True,
#             executions_per_trial=2)

In [12]:
Early = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
Checkpoint = keras.callbacks.ModelCheckpoint("flowersModel.h5", save_best_only=True)

In [13]:
model.fit(train_set,
            validation_data=val_set,
            epochs=22,
            callbacks=[Early],
#             steps_per_epoch=int(0.75 * DATASET_SIZE / BATCH_SIZE),
#             validation_steps=int(0.15 * DATASET_SIZE / BATCH_SIZE),
            )

Epoch 1/22
786/786 [==============================] - 214s 137ms/step - loss: 0.8687 - accuracy: 0.8140 - val_loss: 0.1437 - val_accuracy: 0.9940
Epoch 2/22
786/786 [==============================] - 214s 137ms/step - loss: 0.1486 - accuracy: 0.9882 - val_loss: 0.0816 - val_accuracy: 0.9978
Epoch 3/22
786/786 [==============================] - 219s 143ms/step - loss: 0.0920 - accuracy: 0.9938 - val_loss: 0.0570 - val_accuracy: 0.9993
Epoch 4/22
786/786 [==============================] - 223s 147ms/step - loss: 0.0661 - accuracy: 0.9970 - val_loss: 0.0437 - val_accuracy: 0.9993
Epoch 5/22
786/786 [==============================] - 223s 150ms/step - loss: 0.0538 - accuracy: 0.9979 - val_loss: 0.0367 - val_accuracy: 0.9993
Epoch 6/22
786/786 [==============================] - 221s 142ms/step - loss: 0.0460 - accuracy: 0.9975 - val_loss: 0.0315 - val_accuracy: 0.9993
Epoch 7/22
786/786 [==============================] - 226s 145ms/step - loss: 0.0404 - accuracy: 0.9975 - val_loss: 0.0270 -

In [ ]:
model.evaluate(test_set)

12/86 [===>..........................] - ETA: 10s - loss: 0.0622 - accuracy: 0.9948